In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [2]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

CPU times: user 4.48 s, sys: 3.4 s, total: 7.88 s
Wall time: 31.7 s
done loading pickles (5787634, 34)
(156558, 34)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,plos_article_type,num_cit_young_ref_by2009,num_cit_young_ref_by2010,num_cit_young_ref_by2011,num_cit_young_ref_by2012,num_cit_young_ref_by2013,num_cit_young_ref_by2009after8,num_cit_young_ref_by2008after8,num_cit_young_ref_by2010after8,num_cit_young_ref_by2007after8
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A1995QY75100004,1,3,60.0,1995.0,2,2013.0,494,5398,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,000263911400006,3,0,5.0,2009.0,2,2013.0,142,4029,...,@ Article,0.0,0.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0
5,1,000289279600018,4,0,29.0,2011.0,2,2013.0,269,4029,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0
6,3,000289279600018,4,3,29.0,2011.0,2,2013.0,3844,5398,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0


In [3]:
 ### DONE.  MULTIPLOT FOR BARPLOTS OF NUMBER OF RECORDS AND NUMBER OF PAPERS PER JOURNAL

    
### FIGURE 1A
    
    ### NOTE: THIS FIGURE REQUIRES HEAVY TOUCH-UPS ON INKSCAPE (FOR THE BRONEN Y AXIS)
    
    
v1_string =  'cite_count'#  log2_num_cit_ref'  ######  log2_num_cit_ref' # #     #'ref_pub_year'     cite_count    diff_year_plos_ref 
       

years=[2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017] 



    
string_filtering_x = 'paper_cite_count'   # bins by plos' citations on the x-axis ###      
  
string_references_age = ""   #young"#old"  # young # all   for the selection of what references i include
    
string_isolated_ref = ""   #""   #"  #""   # 0  or 1 (or empty string, to include all ref)
  
string_self_ref =0    #""      # ""   #1   # 0  or 1 (or empty string, to include all ref)




######### plos ONE categories. 
string_code_categ="" #  ojo!!! the codes are strings, not integers. if i want to include multiple subjects:  "1 2 8"
 
#  '0': 'Biology and life sciences'             6,032,537
#  '1': 'Computer and information sciences'     1,207,799
#  '10': 'Social sciences'                      755,899
#  '2': 'Earth sciences'                        533,155
#  '3': 'Ecology and environmental sciences'    624,142
#  '4': 'Engineering and technology'            382,247 
#  '5': 'Medicine and health sciences'          4,535,926   
#  '6': 'People and places'                     691,523
#  '7': 'Physical sciences'                     2,100,827
#  '8': 'Research and analysis methods'         3,871,470
#  '9': 'Science policy'                        43,360 


######### plos journals 
string_journal=""

    # PLOS ONE       6,367,070
    # PLOS GENET      149,923
    # PLO NE TR D     138,289   # (neglected tropical diseases)
    # PLOS PATHOG     109,803
    # PLOS COMPUT      77,924
    # PLOS BIOL        56,754
    # PLOS MED         24,506

 

######### WoS subject categories. 
string_plos_field=""#['D CU BIOLOGY']"

# ['D RO MULTIDISCIPLINARY SCIENCES']                                                                                                       4464540
# ['D CU BIOLOGY']                                                                                                                          1055045
# ['D RO MULTIDISCIPLINARY SCIENCES', 'D CU BIOLOGY']                                                                                        847485
# ['D KM GENETICS & HEREDITY']                                                                                                               149923
# ['D YU TROPICAL MEDICINE', 'D TI PARASITOLOGY']                                                                                            138289
# ['D ZE VIROLOGY', 'D QU MICROBIOLOGY', 'D TI PARASITOLOGY']                                                                                109803
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY']                                                          77687
# ['D CQ BIOCHEMISTRY & MOLECULAR BIOLOGY', 'D CU BIOLOGY']                                                                                   56754
# ['D PY MEDICINE, GENERAL & INTERNAL']                                                                                                       24506
# ['D CO BIOCHEMICAL RESEARCH METHODS', 'D MC MATHEMATICAL & COMPUTATIONAL BIOLOGY', 'D PO MATHEMATICS, INTERDISCIPLINARY APPLICATIONS']        237








      
    
    
print ("original size:",df_merged.shape)





    


##### preselection by plos year
print (years)
preselection_df = df_merged[df_merged['plos_pub_year'].isin(years)]  
print ("size of preselection1 (by plos years):",preselection_df.shape)





#### i remove self-citations
if (string_self_ref==0) or  ( string_self_ref == 1 ): 
    preselection_df = preselection_df[preselection_df['self_citation']== string_self_ref ]  
    if string_self_ref ==0:
        string_self_ref = ", no self-cit"
    elif string_self_ref ==1:
        string_self_ref = ", only self-cit"



        
        
######### preselection by isolated or group references:
if (string_isolated_ref==0) or  ( string_isolated_ref == 1 ): 
    preselection_df0 = preselection_df[preselection_df['isolated_citation']== string_isolated_ref ]  
    
    if string_isolated_ref ==0:
        string_isolated_ref = ", group ref"
    elif string_isolated_ref ==1:
        string_isolated_ref = ", isolated ref"
else:    
    preselection_df0 = preselection_df   
    print ("size of preselection1 (by isolated/group ref):",preselection_df0.shape, string_isolated_ref)




    
    
    
    
######### preselection by plos ONE subject category:
if string_code_categ=="": 
    preselection_df111 = preselection_df0
else:    
    if " " not in string_code_categ:  # to include one single category
        preselection_df111 = preselection_df0[preselection_df0['categ_codes'].str.contains(string_code_categ)]        
        string_code_categ = " "+dict_code_categ[string_code_categ]  
        
    else:  # if multiple codes-categories
        list_codes = string_code_categ.split(" ")
        print (list_codes)

        if len(list_codes) >= 2:              
            preselection_df111 = preselection_df0[ preselection_df0['categ_codes'].str.contains('|'.join(list_codes)) ]  # to look for partial matches from a list of strings!!!!!
            
       
        string_code_categ = "" 
        for code in list_codes:
            string_code_categ += "-"+dict_code_categ[code] 
            
            
    print (" size of preselection (by plos ONE subject category):",preselection_df111.shape, string_code_categ)
           
        
        
        
        
        
        

######### preselection by plos journal:
if string_journal=="": 
    preselection_df1 = preselection_df111
else:    
    preselection_df1 = preselection_df111[preselection_df111['plos_j1']== string_journal ]  
print (" size of preselection2 (by plos journal):",preselection_df1.shape, string_journal)







######### preselection by plos field:
if string_plos_field=="": 
    preselection_df2 = preselection_df1
else:    
    preselection_df2 = preselection_df1[preselection_df1['plos_field']== string_plos_field ]  
print (" size of preselection2 (by plos field):",preselection_df2.shape, string_plos_field)

string_references_age


preselection_df3 = preselection_df2
   
  



if v1_string ==  'cite_count'  or       v1_string ==  'log_num_cit_ref'   or v1_string == 'log2_num_cit_ref':
     
      
    
    string_age_selection=''

    ##### preselection only young/old references:        
    if string_references_age == "young":
        time_window = 1
        string_age_selection="only young references from >="+ str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] >= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape, string_age_selection)
        
    elif string_references_age == "old":
        time_window = 10
        string_age_selection="only old references from <="+str((min(years)-time_window))
        preselection_df3 = preselection_df2[preselection_df2['ref_pub_year'] <= (min(years)-time_window) ]   
        print ("  size of preselection3 (only young references):",preselection_df3.shape,string_age_selection )
        
    else:
        string_age_selection="young&old"       
        print ("  No preselection by age of references:",preselection_df3.shape )
          
    
    

N_plos=len(preselection_df3.paper_UT.unique())        ## this values are overall, for the title   
N_all=len(preselection_df3)
 

print ("\nTot # records included:",len(preselection_df3),"   # number of plos papers:",len(preselection_df3.paper_UT.unique()), "   # unique ref:", len(preselection_df3.reference_UT.unique()),'\n')

 
    
    
    
    
    
    
    
    
   
list_colors = ["#FF4040","#4EEE94","#87CEFA","#FFA500","#EE82EE","#8B8B83", "#0000FF"]
list_journals = ['PLOS GENET', 'PLO NE TR D','PLOS PATHOG', 'PLOS COMPUT', 'PLOS BIOL',  'PLOS MED', 'PLOS ONE']




fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.001, horizontal_spacing=0.001)






data = []

list_num_papers = []
list_num_records = []
cont = 0
for journal in list_journals:
    df_selection = preselection_df3[preselection_df3['plos_j1'] == journal]        
    
    trace1 = go.Bar(
        x=['# papers'],
        y=[ len(df_selection.paper_UT.unique())],
        marker=dict( color=list_colors[cont]),
        #name=journal.replace('PLO NE TR D','PLOS Neglected<br>  Tropical Diseases').replace('PLOS BIOL','PLOS Biology').replace('PLOS COMPUT','PLOS Computational<br>  Biology').replace('PLOS GENET','PLOS Genetics').replace('PLOS MED','PLOS Medicine').replace('PLOS PATHOG','PLOS Pathology'),        
        name=journal.replace('PLO NE TR D','Neglected<br>  Tropical Diseases').replace('PLOS BIOL','Biology').replace('PLOS COMPUT','Computational<br>  Biology').replace('PLOS GENET','Genetics').replace('PLOS MED','Medicine').replace('PLOS PATHOG','Pathology'),
        showlegend=True
    )
   
    fig.append_trace(trace1, 1, 1)
    cont +=1


    
    
cont = 0    
for journal in list_journals:
    df_selection = preselection_df3[preselection_df3['plos_j1'] == journal]
    
    
    trace2 = go.Bar(
        x=['# records'],   
        y=[len(df_selection)],
        marker=dict( color=list_colors[cont]),
        #name=journal.replace('PLO NE TR D','PLOS Neglected\n Tropical Diseases').replace('PLOS BIOL','PLOS Biology').replace('PLOS COMPUT','PLOS Computational Biology').replace('PLOS GENET','PLOS Genetics').replace('PLOS MED','PLOS Medicine').replace('PLOS PATHOG','PLOS Pathology'),
        showlegend=False,
        xaxis='x2',
        yaxis='y2'
    )
    fig.append_trace(trace2, 1, 2)
    cont  +=1

 


#fig['layout'].update(height=400, width=1200, legend=dict(orientation="h"), margin=go.Margin( l=200))#, b=150),)# r=50, b=150,  #t=100, # pad=4 ), )
fig['layout'].update( legend=dict(x=1.2, y=1.0), margin=go.Margin( l=250))#, b=150),)# r=50, b=150,  #t=100, # pad=4 ), )



fig['layout'].update(barmode='stack')


fig['layout']['yaxis2'].update(side='right')
fig['layout']['yaxis'].update(title='Count')

# fig['layout']['yaxis'].update(range=[0, 15000])  
# fig['layout']['yaxis2'].update(range=[0, 800000])  


#fig['layout']['yaxis'].update(type='log')
#fig['layout']['yaxis2'].update(type='log')
# fig['layout']['yaxis'].update(range=[0, 4])  ### ojo!! rango logaritmico if axis in log type!  0, 5.3
# fig['layout']['yaxis'].update(range=[0, 6.84])  ### ojo!! rango logaritmico!   0, 6.84





font_gral=55 # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# Altering axis
fig['layout']['xaxis']['tickfont']['size']  = font_gral + 10
fig['layout']['xaxis2']['tickfont']['size'] = font_gral + 10
fig['layout']['yaxis']['tickfont']['size']  = font_gral + 10
fig['layout']['yaxis2']['tickfont']['size'] = font_gral + 10



fig['layout']['xaxis'].update(domain=[0, 0.45]) # this is to force the relative location of the two panels with respect to each other
fig['layout']['yaxis'].update(domain=[0, 1])

fig['layout']['xaxis2'].update( domain=[0.55, 1])
fig['layout']['yaxis2'].update(domain=[0, 1],  anchor='x2')








offline.plot(fig, auto_open=True, image = 'svg', image_filename="fig1a" ,image_width=2000, image_height=1000, 
              filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/fig1a', validate=True)







original size: (5787630, 34)
[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
size of preselection1 (by plos years): (5787630, 34)
size of preselection1 (by isolated/group ref): (5787630, 34) 
 size of preselection2 (by plos journal): (5787630, 34) 
 size of preselection2 (by plos field): (5787630, 34) 
  No preselection by age of references: (5787630, 34)

Tot # records included: 5787630    # number of plos papers: 156558    # unique ref: 2320774 

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



/home/staff/julia/.pyenv/versions/3.6.4/envs/psy/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:396: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin


/home/staff/julia/.pyenv/versions/3.6.4/envs/psy/lib/python3.6/site-packages/plotly/offline/offline.py:463: UserWarning:

Your filename `/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/fig1a` didn't end with .html. Adding .html to the end of your file.



'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/fig1a.html'